# Influxdb 数据连接及查询指导

1. 确认连接信息

In [1]:
# 确认连接信息
DB_HOST = '192.168.3.94'
DB_PORT = '8086'
DB_USER = 'admin'
DB_PASSWD = '123456'
# 注意点， 每次连接的数据库名很重要。
# 连接某一数据库， 那么将只能对此数据库下的measurements进行操作。
DB_NAME = 'PLC_DATABASE'

## 导入连接InfluxDB python连接包并连接

目前InfluxDB-Python包中，有以下三种连接方式：
1. InlfuxDBClient ---> 主要连接方式，数据格式自定义，灵活多变，限制底。
2. DataFrameClient ---> 基于InfluxDBClient连接，扩展对Pandas Dataframe数据格式的操作。
3. SeriesHelper ---> 主要用于辅助对于数据的写入。

### 以下为InfluxDBClient连接示例

In [2]:
from influxdb import InfluxDBClient

# 创建连接对象
client = InfluxDBClient(DB_HOST, DB_PORT, DB_USER, DB_PASSWD, DB_NAME)
client

**以下是查询所有字段， 来自T41 measurements， 并限制为5条**

measurement 可以理解为SQL中的**表**概念。

由此看着数据较为混乱，没有较好的序列化展示，可**操作性**较为复杂等。

所以不推荐使用，以下将**主要**讲述使用DataFrame 连接方式， 获取较为直接的pandas Dataframe 数据。

In [3]:
result = client.query("select * from T41 limit 5")
result

ResultSet({'('T41', None)': [{'time': '2014-01-18T00:00:00Z', '$Date': '01/18/14', '$Time': '00:00:00', 'BARE-FIB-DIAMETER': 124.4949, 'BFIB#FFIBERTENSIONMAXERR': 0, 'BFIB#FXPOSITIONAVG': 20, 'BFIB#FXPOSITIONME': 560.6229, 'BFIB#FYPOSITIONAVG': 20, 'BFIB#FYPOSITIONME': 288.3483, 'COAT#FBODYTEMPME': 2.5, 'COAT#INPRIMCTTEMP': 48.93564, 'COAT-PIPEPWMAS': 0.0536931, 'COAT-PRIMPIPETEMP': 0, 'COAT-SECOPIPETEMP': 47.93521, 'COPI#INUPPERHEFLOW': 3.881954, 'CPST#MSP1KD': 0, 'CPST#MSP1KI': 0, 'CPST#MSP1KP': 0, 'CTFB#NDIAMETERPUFFDISPLAY': 40, 'DRUM-LENGTH': 289974, 'FIBER-TENSION': 166.2366, 'FIBER-TOTAL-LENGTH': 571592, 'FURN#FBODYPRESSME': 0, 'FURN#FPOWERAS': 50.64008, 'FURN-ELEMENT-AR-FLOW': 15.06042, 'FURN-ELEMENT-HE-FLOW': 10.02018, 'FURN-POWER-ME': 50.63984, 'FURN-TEMP': 1842.0710000000001, 'FURN-TOPIRIS-AR-FLOW': 15.060089999999999, 'LINE-SPEED': 1571.0510000000002, 'PREF#FPREFORMCSACAL': 0, 'PREF#FXAXISDRIVESPEEDAS': 0, 'PREF#FYAXISDRIVESPEEDAS': 0, 'PREF-FEED-SPEED': 0.625, 'PREF-FEEDER

### 以下为使用DataFrameClient 连接示例

In [4]:
from influxdb import DataFrameClient

# 创建连接对象
pd_client = DataFrameClient(DB_HOST, DB_PORT, DB_USER, DB_PASSWD, DB_NAME)
pd_client

**以下是查询所有字段， 来自T41 measurements， 并限制为5条**

由此结果可直接看出较好的直观性，使用方便。

查询的结果将返回一个以measurements name 为键， 数据为值的字典，只需要使用字典的获取方式，获取measurements name的值即可。

就是可直接操作的DataFrame对象。默认index为时间戳。

In [5]:
df = pd_client.query("select * from T41 limit 5")
df

defaultdict(list,
            {'T41':                               $Date     $Time  BARE-FIB-DIAMETER  \
             2014-01-18 00:00:00+00:00  01/18/14  00:00:00           124.4949   
             2014-01-18 00:00:01+00:00  01/18/14  00:00:01           124.4949   
             2014-01-18 00:00:02+00:00  01/18/14  00:00:02           124.4286   
             2014-01-18 00:00:03+00:00  01/18/14  00:00:03           124.5040   
             2014-01-18 00:00:04+00:00  01/18/14  00:00:04           124.4223   
             
                                        BFIB#FFIBERTENSIONMAXERR  BFIB#FXPOSITIONAVG  \
             2014-01-18 00:00:00+00:00                         0                  20   
             2014-01-18 00:00:01+00:00                         0                  20   
             2014-01-18 00:00:02+00:00                         0                  20   
             2014-01-18 00:00:03+00:00                         0                  20   
             2014-01-18 00:00:04+00

In [6]:
df['T41']

,$Date,$Time,BARE-FIB-DIAMETER,BFIB#FFIBERTENSIONMAXERR,BFIB#FXPOSITIONAVG,BFIB#FXPOSITIONME,BFIB#FYPOSITIONAVG,BFIB#FYPOSITIONME,COAT#FBODYTEMPME,COAT#INPRIMCTTEMP,...,PREF-FEED-SPEED,PREF-FEEDER-POSITION,PREF-RELATIVE-POS,PREF-VACUUM,PRI-COAT-CAPRESS-SP,PRI-COAT-PRESSURE,SEC-COAT-CAPRESS-SP,SEC-COAT-PRESSURE,SEC-FIB-DIAMETER,dt_str
2014-01-18 00:00:00+00:00,01/18/14,00:00:00,124.4949,0,20,560.6229,20,288.3483,2.5,48.93564,...,0.625,1604.59,307.39,10,3.971051,3.564079,4.542102,3.987772,241.1610,01/18/14 00:00:00
2014-01-18 00:00:01+00:00,01/18/14,00:00:01,124.4949,0,20,560.6229,20,288.3483,2.5,48.93564,...,0.625,1604.59,307.39,10,3.971051,3.564079,4.542102,3.987772,241.1610,01/18/14 00:00:01
2014-01-18 00:00:02+00:00,01/18/14,00:00:02,124.4286,0,20,556.0009,20,287.4855,2.5,48.93564,...,0.625,1604.60,307.40,10,3.970916,3.559863,4.541832,3.986936,241.3838,01/18/14 00:00:02
2014-01-18 00:00:03+00:00,01/18/14,00:00:03,124.5040,0,20,566.2587,20,165.3035,2.5,48.93865,...,0.625,1604.61,307.41,10,3.970683,3.558442,4.541366,3.986417,241.4203,01/18/14 00:00:03
2014-01-18 00:00:04+00:00,01/18/14,00:00:04,124.4223,0,20,365.3734,20,127.5438,2.5,48.93865,...,0.625,1604.62,307.42,10,3.970419,3.559237,4.540839,3.987446,241.3300,01/18/14 00:00:04


In [7]:
df['T41'].index

DatetimeIndex(['2014-01-18 00:00:00+00:00', '2014-01-18 00:00:01+00:00',
               '2014-01-18 00:00:02+00:00', '2014-01-18 00:00:03+00:00',
               '2014-01-18 00:00:04+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### 以下主要说明查询语句的使用以及案例

**查询一个measurements 的所有字段数据** 

其中的 **T41** 为需要查询的measurement name 也可以理解为SQL中的**表**概念，

谨慎执行。

In [8]:
# all_fields = pd_client.query('select * from "T41"')
# all_fields

**查询特定的tag和field**

谨慎执行。

limit 为限制查询结果的返回条数。

以下为查询 "$Date","$Time" 两个字段的值， 来自T41， 限制数据量为10000条。

In [14]:
tag_or_fields = pd_client.query('select "$Date","$Time" from "T41" limit 10000')
tag_or_fields['T41']

,$Date,$Time
2014-01-18 00:00:00+00:00,01/18/14,00:00:00
2014-01-18 00:00:01+00:00,01/18/14,00:00:01
2014-01-18 00:00:02+00:00,01/18/14,00:00:02
2014-01-18 00:00:03+00:00,01/18/14,00:00:03
2014-01-18 00:00:04+00:00,01/18/14,00:00:04
2014-01-18 00:00:05+00:00,01/18/14,00:00:05
2014-01-18 00:00:06+00:00,01/18/14,00:00:06
2014-01-18 00:00:07+00:00,01/18/14,00:00:07
2014-01-18 00:00:08+00:00,01/18/14,00:00:08
2014-01-18 00:00:09+00:00,01/18/14,00:00:09


**注意点**

**一个查询在`SELECT`子句中至少需要一个field key来返回数据。如果`SELECT`子句仅包含单个tag key或多个tag key，则查询返回一个空的结果。**

此示例measurements 为全部的fields字段， 所以无需注意。

**使用where查询特定field字段的特定值的数据**

以上为查询字段 “LINE-SPEED” 等于 1571等数据， 返回条数为100。
另外支持 fields类型以下运算：

 `=` 等于；
 `<>` 不等于；
 `!=` 不等于；
 `>` 大于；
 `>=` 大于等于；
 `<` 小于；
 `<=` 小于等于；

tags类型以下运算：

 `=` 等于；
 `<>` 不等于；
 `!=` 不等于；

注意点： 注意对比的数据类型， 例：字符串支持持等于和不等于。

In [19]:
where_equal = pd_client.query('select * from "T41" where "PREF-VACUUM" = 10 limit 10')
where_equal['T41']

,$Date,$Time,BARE-FIB-DIAMETER,BFIB#FFIBERTENSIONMAXERR,BFIB#FXPOSITIONAVG,BFIB#FXPOSITIONME,BFIB#FYPOSITIONAVG,BFIB#FYPOSITIONME,COAT#FBODYTEMPME,COAT#INPRIMCTTEMP,...,PREF-FEED-SPEED,PREF-FEEDER-POSITION,PREF-RELATIVE-POS,PREF-VACUUM,PRI-COAT-CAPRESS-SP,PRI-COAT-PRESSURE,SEC-COAT-CAPRESS-SP,SEC-COAT-PRESSURE,SEC-FIB-DIAMETER,dt_str
2014-01-18 00:00:00+00:00,01/18/14,00:00:00,124.4949,0,20,560.6229,20,288.348300,2.5,48.93564,...,0.625,1604.59,307.39,10,3.971051,3.564079,4.542102,3.987772,241.1610,01/18/14 00:00:00
2014-01-18 00:00:01+00:00,01/18/14,00:00:01,124.4949,0,20,560.6229,20,288.348300,2.5,48.93564,...,0.625,1604.59,307.39,10,3.971051,3.564079,4.542102,3.987772,241.1610,01/18/14 00:00:01
2014-01-18 00:00:02+00:00,01/18/14,00:00:02,124.4286,0,20,556.0009,20,287.485500,2.5,48.93564,...,0.625,1604.60,307.40,10,3.970916,3.559863,4.541832,3.986936,241.3838,01/18/14 00:00:02
2014-01-18 00:00:03+00:00,01/18/14,00:00:03,124.5040,0,20,566.2587,20,165.303500,2.5,48.93865,...,0.625,1604.61,307.41,10,3.970683,3.558442,4.541366,3.986417,241.4203,01/18/14 00:00:03
2014-01-18 00:00:04+00:00,01/18/14,00:00:04,124.4223,0,20,365.3734,20,127.543800,2.5,48.93865,...,0.625,1604.62,307.42,10,3.970419,3.559237,4.540839,3.987446,241.3300,01/18/14 00:00:04
2014-01-18 00:00:05+00:00,01/18/14,00:00:05,124.5462,0,20,395.0784,20,1.362834,2.5,48.93904,...,0.625,1604.63,307.43,10,3.970217,3.559867,4.540433,3.987589,241.1366,01/18/14 00:00:05
2014-01-18 00:00:06+00:00,01/18/14,00:00:06,124.5118,0,20,333.6735,20,128.771200,2.5,48.93904,...,0.625,1604.64,307.44,10,3.970047,3.561421,4.540093,3.988118,241.1971,01/18/14 00:00:06
2014-01-18 00:00:07+00:00,01/18/14,00:00:07,124.5378,0,20,356.2085,20,236.914100,2.5,48.94033,...,0.625,1604.65,307.45,10,3.969824,3.557199,4.539649,3.985919,241.4909,01/18/14 00:00:07
2014-01-18 00:00:08+00:00,01/18/14,00:00:08,124.5772,0,20,468.7034,20,10.156200,2.5,48.94033,...,0.625,1604.66,307.46,10,3.969785,3.557527,4.539570,3.986054,241.6458,01/18/14 00:00:08
2014-01-18 00:00:09+00:00,01/18/14,00:00:09,124.6501,0,20,455.4063,20,412.585400,2.5,48.94115,...,0.625,1604.68,307.48,10,3.969831,3.560491,4.539662,3.988033,241.1709,01/18/14 00:00:09


**where条件支持  时间  查询**

该查询返回measurement 大于**现在时间减去1000天的时间**， 在过去其他的数据，可以使用长天数， 例：1200d(天),1200m(分钟)。 但是同样的年月写法， 错误写法：120M， 10Y。

In [13]:
where_time = pd_client.query('select * from "T41" where time > now() -1000d limit 100')
where_time['T41']

defaultdict(list,
            {'T41':                               $Date     $Time  BARE-FIB-DIAMETER  \
             2016-02-18 00:00:00+00:00  02/18/16  00:00:00                120   
             2016-02-18 00:00:01+00:00  02/18/16  00:00:01                120   
             2016-02-18 00:00:02+00:00  02/18/16  00:00:02                120   
             2016-02-18 00:00:03+00:00  02/18/16  00:00:03                120   
             2016-02-18 00:00:04+00:00  02/18/16  00:00:04                120   
             2016-02-18 00:00:05+00:00  02/18/16  00:00:05                120   
             2016-02-18 00:00:06+00:00  02/18/16  00:00:06                120   
             2016-02-18 00:00:07+00:00  02/18/16  00:00:07                120   
             2016-02-18 00:00:08+00:00  02/18/16  00:00:08                120   
             2016-02-18 00:00:09+00:00  02/18/16  00:00:09                120   
             2016-02-18 00:00:10+00:00  02/18/16  00:00:10                120   
   

**group by 分组查询**

主要描述根据时间分组的情况。

tags同样可以作为分组条件，但是fields不行。

由于此数据信息没有tags， 不做演示。

**注意点**: 查询语句中的时间戳只能使用**单引号**， 使用双引号会出现以下错误：

invalid operation: time and *influxql.VarRef are not compatible

In [45]:
ql = 'select count("COAT#INPRIMCTTEMP") from "T41" where "PREF-RELATIVE-POS"=307.39 and time >= \'2016-02-18T00:00:00Z\' and time <= \'2016-02-19T00:00:00Z\' group by time(30m)'

group_time = pd_client.query(ql)

group_time['T41']

,count
2016-02-18 00:00:00+00:00,0
2016-02-18 00:30:00+00:00,0
2016-02-18 01:00:00+00:00,0
2016-02-18 01:30:00+00:00,0
2016-02-18 02:00:00+00:00,0
2016-02-18 02:30:00+00:00,0
2016-02-18 03:00:00+00:00,0
2016-02-18 03:30:00+00:00,0
2016-02-18 04:00:00+00:00,0
2016-02-18 04:30:00+00:00,0
